In [1]:
from dolfinx.mesh import create_rectangle
from mpi4py import MPI

from dolfinx.fem import function, functionspace
from basix.ufl import element, mixed_element


In [2]:
from MultiphysicsModel.Mesh import RBMesh
import numpy as np

a = 1.44e-3

model_parameters = {
    "a":            a,
    "coords":       ((-2*a,-2*a),(2*a,2*a)),
    "grid_size":    (100,100),
    "temp_top":     291.152, # K
    "temp_bottom":  290.000, # K
}


markers = {
    "leftright":    lambda x: np.isclose(x[0], -2*a) | np.isclose(x[0],2*a),
    "top":          lambda x: np.isclose(x[1], 2*a),
    "bottom":       lambda x: np.isclose(x[1], -2*a),
}

material_model = {
    "alpha1": {
        "rho":      250.0,
        "cp":       5e-5,
        "mu":       0.012,
        "Pr":       0.5,
    },
    "alpha2": {
        "rho":      500.0,
        "cp":       1e-4,
        "mu":       0.024,
        "Pr":       1.0,
    },
}

mesh = RBMesh(points=model_parameters["coords"], 
              n=model_parameters["grid_size"],
              bc_markers=markers)

In [3]:
from MultiphysicsModel.FEData import RBData

fe_config = {

    "alpha1": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
    },

    "alpha2": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "p": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "u": {
        "element": "CG", 
        "degree": 1, 
        "type": "vector",
        "time_scheme":  "explicit euler",
        },

    "T": {
        "element": "CG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },
}

fe_data = RBData(mesh=mesh, config=fe_config, create_mixed=True)

According to [this discussion](https://fenicsproject.discourse.group/t/cannot-find-dofs-for-discontinuous-element/10900/3), discontinuous spaces do not have DoFs that live on a facet. Thus, `locate_dofs_topological` does not find any DoFs there. Must rely on `locate_dofs_geometrical` for now.

In [4]:
from MultiphysicsModel.InitialCondition import RBInitialConditions

ics = RBInitialConditions(params=model_parameters)
ics.apply(fe_data=fe_data)

In [5]:
from MultiphysicsModel.BoundaryCondition import RBBoundaryConditions

bcs = RBBoundaryConditions(fe_data=fe_data,parameters=model_parameters)
bcs.apply(mesh=mesh,fe_data=fe_data)

In [6]:
from MultiphysicsModel.FEOperators import FEOperators

ops = FEOperators(mesh=mesh)

In [6]:
from MultiphysicsModel.RisingBubbleModel import RisingBubbleModel

model = RisingBubbleModel(
    model_parameters=model_parameters,
    material_model=material_model,
    fe_config=fe_config,
    bc_markers=markers,
    timestep=2e-5,
    time_domain=(0.0,0.12),
    create_mixed=True
)

Model has Degrees of Freedom (DoFs):
alpha1:    60,000
alpha2:    60,000
p:    60,000
u:    20,402
T:    10,201
Total:    210,603
